In [1]:
#If run through Docker we'll point at the local 'unsupported dir.'
#On hpc, the path should also include a directory holding the unsupported_dan.
import os
import sys

#this does't actually need to be protected. More a reminder it's an interim measure
try:
    sys.path.append('../../unsupported')
except:
    pass

In [2]:
#from unsupported_dan.UWsubduction.base_params import *;

from easydict import EasyDict as edict
import unsupported_dan.geodynamics.scaling as sca;
import pickle
import pint

../../unsupported/unsupported_dan/geodynamics/__init__.py:21: UserWarning: 

The geodynamics module is deprecated.
Please use UWGeodynamics instead.

You can install the module using:


```
    pip install UWGeodynamics
```

You can also use the docker image:


rbeucher/underworld_geodynamics


Finally on github:


https://github.com/rbeucher/UWGeodynamics


  """


In [3]:
#from unsupported_dan.UWsubduction.base_params import dimParams as dp
#from unsupported_dan.UWsubduction.base_params import modelParams as md

In [4]:
u = sca.UnitRegistry

In [5]:
#u.km
#u.delta_degC.dimensionless


In [12]:
dp = edict({})

#Main physical paramters (thermal convection parameters)
dp.refDensity = 3300.* u.kilogram / u.meter**3                    #reference density
dp.refGravity = 9.8* u.metre / u.second**2                        #surface gravity
dp.refDiffusivity = 1e-6 *u.metre**2 / u.second                   #thermal diffusivity
dp.refExpansivity = 3e-5/u.kelvin                                        #surface thermal expansivity
dp.refViscosity = 1e20* u.pascal* u.second
dp.refLength = 2900*u.km
dp.gasConstant = 8.314*u.joule/(u.mol*u.kelvin)                   #gas constant
dp.specificHeat = 1250.4*u.joule/(u.kilogram* u.kelvin)           #Specific heat (Jkg-1K-1)
dp.potentialTemp = 1573.*u.kelvin                                 #mantle potential temp (K)
dp.surfaceTemp = 273.*u.kelvin                                    #surface temp (K)
#dp.deltaT = dp.potentialTemp - dp.surfaceTemp
#main rheology paramters (thermal convection parameters)
dp.cohesionMantle = 20.*u.megapascal                              #mantle cohesion in Byerlee law
dp.frictionMantle = u.Quantity(0.2)                                           #mantle friction coefficient in Byerlee law (tan(phi))
dp.frictionMantleDepth = dp.frictionMantle*dp.refDensity*dp.refGravity
dp.diffusionPreExp = 5.34e-10/u.pascal/u.second                   #pre-exp factor for diffusion creep
dp.diffusionEnergy = 3e5*u.joule/(u.mol)
dp.diffusionEnergyDepth = 3e5*u.joule/(u.mol*dp.gasConstant)
dp.diffusionVolume=5e-6*u.meter**3/(u.mol)
dp.diffusionVolumeDepth=5e-6*dp.refDensity.magnitude*dp.refGravity.magnitude*u.joule/(u.mol*dp.gasConstant*u.meter)
dp.viscosityInterface = 5e19*u.pascal   * u.second
dp.adiabaticTempGrad = (dp.refExpansivity*dp.refGravity*dp.potentialTemp)/dp.specificHeat


md = edict({})
#Model geometry, and misc Lengths used to control behaviour
md.depth=1000*u.km                                                #Model Depth
md.aspectRatio=5.
md.interfaceViscCutoffDepth = 100*u.km 
md.interfaceViscEndWidth = 20*u.km
md.faultThickness = 10.*u.km  
md.faultCleanUpDepth = 500*u.km                                    #interface material (crust) an top of slabs
md.lowerMantleDepth=660.*u.km 
#Slab and plate init. parameters
md.subZoneLoc=-100*u.km                                           #X position of subduction zone...km
md.slabInitMaxDepth=150*u.km
md.radiusOfCurv = 350*u.km                                        #radius of curvature
md.slabAge=70*u.megayears                                      #age of subduction plate at trench
md.opAgeAtTrench=35*u.megayears                                        #age of op
#numerical and computation params
md.res=48
md.ppc=25                                                         #particles per cell
md.elementType="Q1/dQ0"
md.nltol = 0.01
md.druckerAlpha = 0.
md.penaltyMethod = True
md.buoyancyFac = 1.0
md.faultLocFac = 1. #this is the relative location of the fault in terms of the fault thickess from the top of slab
md.viscosityMin = 1e18* u.pascal * u.second
md.viscosityMax = 1e25* u.pascal * u.second
md.lowerMantleViscFac = 30.0
md.yieldStressMax=200*u.megapascal


In [13]:
#dp.refExpansivity, dp.refGravity, dp.potentialTemp, dp.specificHeat
#sca.nonDimensionalize(dp.adiabaticTempGrad)
dp.adiabaticTempGrad, sca.nonDimensionalize(dp.adiabaticTempGrad)

(<Quantity(0.000369851247601, 'kelvin * kilogram * meter / joule / second ** 2')>,
 0.8250527831094052)

In [14]:
dp.adiabaticTempGrad.units == u.kelvin


False

In [15]:
#Define scales used to non-dimensionalize the system

KL = dp.refLength
KT = dp.potentialTemp - dp.surfaceTemp
Kt = KL**2/dp.refDiffusivity
KM = dp.refViscosity * KL * Kt

sca.scaling["[length]"]      = KL.to_base_units()
sca.scaling["[temperature]"] = KT.to_base_units()
sca.scaling["[mass]"]        = KM.to_base_units()
sca.scaling["[time]"] =        Kt.to_base_units()

In [16]:
tempOffset = pint.converters.OffsetConverter(1, -273*u.kelvin )

ndp = edict({})
for key, val in dp.items():
    if val.unitless:
        ndp[key] = val.magnitude
    elif val.units == u.kelvin:
        ndp[key] = sca.nonDimensionalize(tempOffset.to_reference(val))
    else:
        ndp[key] = sca.nonDimensionalize(val)
    
#For the activation Enthalpy, we are goinf to alter the scaling, 
#effectively incorporatting the gas constant into the Enthalpy,

#ndp.diffusionEnergy /=sca.nonDimensionalize(dp.gasConstant)
#ndp.frictionMantle = dp.frictionMantle/ (dp.refViscosity*dp.refDiffusivity/(dp.refDensity*dp.refGravity*dp.refLength.to_base_units()**3)).magnitude


In [17]:
ndp


{'adiabaticTempGrad': 0.8250527831094052,
 'cohesionMantle': 1682000.0,
 'diffusionEnergy': 1.0344827586206896e-15,
 'diffusionEnergyDepth': 27.546703243833388,
 'diffusionPreExp': 53400000000.0,
 'diffusionVolume': 2.0501045553323221e-25,
 'diffusionVolumeDepth': 43.386502840435966,
 'frictionMantle': 0.2,
 'frictionMantleDepth': 1577480520.0000002,
 'gasConstant': 3.726965517241379e-17,
 'potentialTemp': 1.0,
 'refDensity': 3.2999999999999996e-23,
 'refDiffusivity': 1.0,
 'refExpansivity': 0.039,
 'refGravity': 2.3901220000000004e+32,
 'refLength': 1.0,
 'refViscosity': 0.9999999999999998,
 'specificHeat': 1.36706232e+31,
 'surfaceTemp': 0.0,
 'viscosityInterface': 0.4999999999999999}

In [32]:
nmd = edict({})
for key, val in md.items():
    nmd[key] = sca.nonDimensionalize(val)

In [34]:
nmd.tempGradMantle

0.8250527831094052

In [27]:
rayleighNumber = (dp.refExpansivity*dp.refDensity*dp.refGravity*(dp.potentialTemp - dp.surfaceTemp)*dp.refLength**3).to_base_units()/(dp.refViscosity*dp.refDiffusivity).to_base_units()
assert rayleighNumber.unitless

In [19]:
(dp.refViscosity*dp.refDiffusivity).to_base_units()

<Quantity(1e+14, 'kilogram * meter / second ** 2')>

In [44]:
1./sca.nonDimensionalize(1.*u.pascal)

11.890606420927469

In [18]:
stressScale = ((dp.refDiffusivity*dp.refViscosity)/dp.refLength**2).to_base_units()

<Quantity(11.8906064209, 'kilogram / meter / second ** 2')>

In [21]:
pressureDepthGrad = (dp.refDensity*dp.refGravity*dp.refLength**3)/(dp.refViscosity*dp.refDiffusivity)